In [1]:
# make sure working directory is root of repository
%cd ..
%pwd

/Users/lydiareader/git/dcds-500-project-2


'/Users/lydiareader/git/dcds-500-project-2'

In [2]:
from shopping_environment import ShoppingEnvironment
import pandas as pd

## Setup Environment

In [3]:
# parameters
num_shoes        = 1000

# number of each consumer type
num_average     = 9000
num_abnormal    = 1000
num_special     = 250
num_wealthy     = 1000
num_influencers = 500
num_resellers   = 2500
num_total       = num_average + num_abnormal + num_special + num_wealthy + num_influencers + num_resellers

# initialize environment
env = ShoppingEnvironment(
    mean_desire = 0.5,
    std_desire = 0.2,
    desire_threshold = 0.5,
    average_mean_money = 250,
    average_std_money = 15,
    rich_mean_money = 1000,
    rich_std_money = 250,
    average_prob_loyal = 0.5,
    rich_prob_loyal = 0.8,
    reseller_prob_loyal = 0.9,
    num_shoes = num_shoes,
    price = 250,
    num_average = num_average,
    num_abnormal = num_abnormal,
    num_special = num_special,
    num_wealthy = num_wealthy,
    num_influencers = num_influencers,
    num_resellers = num_resellers
)

In [4]:
def get_utilities(df):
    # buyer and seller utility for each simulation
    got_shoes = df[df['shoes_acquired'] >= 1]
    print("Consumer utility: ", got_shoes['desire'].mean())     # buyer utility is average desire of those who purchased shoes
    print("Seller utility: ", got_shoes['influence'].mean())    # seller utility is average influence of those who purchased shoes 


def group_fairness(df):
    ret_dict = {}
    got_shoes = df[df['shoes_acquired'] >= 1]
    ret_dict['Consumer utility'] = got_shoes['desire'].mean()
    ret_dict['Seller utility'] = got_shoes['influence'].mean()
    amount_con_types = {
        'average': num_average,
        'abnormal': num_abnormal,
        'special': num_special,
        'wealthy': num_wealthy,
        'influencer': num_influencers,
        'reseller': num_resellers
    }
    for con, amount in amount_con_types.items():
        ret_dict[f'{con}_num_got_shoes'] = len(got_shoes[got_shoes['identity'] == con])
        ret_dict[f'{con}_percent_got_shoes'] = len(got_shoes[got_shoes['identity'] == con]) / amount
        ret_dict[f'{con}_shoes_bought'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum()
        ret_dict[f'{con}_shoes_per_person'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum() / amount

    return ret_dict
        

## No Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [5]:
env.restock(1000)
env.run_lottery_no_gaming()  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_no_gaming = pd.Series(group_fairness(df), name='lottery_no_gaming')

### First Come First Served
With no gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [6]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_no_gaming()
df = env.get_consumer_df()
first_no_gaming = pd.Series(group_fairness(df), name='first_no_gaming')

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [7]:
env.restock(num_shoes = 1000)
env.run_invitation_no_gaming()
df = env.get_consumer_df()
invite_no_gaming = pd.Series(group_fairness(df), name='invite_no_gaming')

In [8]:
no_gaming = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming])

## With Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [9]:
env.restock(1000)
env.run_lottery_with_gaming()  # run lottery and get dataframe of all results
df = env.get_fake_df()
lottery_with_gaming = pd.Series(group_fairness(df), name='lottery_with_gaming')

### First Come First Served
With gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [5]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_with_gaming()

df = env.get_fake_df()
first_with_gaming = pd.Series(group_fairness(df), name='first_with_gaming')
first_with_gaming

Consumer utility                  0.411464
Seller utility                    0.328014
average_num_got_shoes           183.000000
average_percent_got_shoes         0.020333
average_shoes_bought            273.000000
average_shoes_per_person          0.030333
abnormal_num_got_shoes           15.000000
abnormal_percent_got_shoes        0.015000
abnormal_shoes_bought            21.000000
abnormal_shoes_per_person         0.021000
special_num_got_shoes             5.000000
special_percent_got_shoes         0.020000
special_shoes_bought             10.000000
special_shoes_per_person          0.040000
wealthy_num_got_shoes            26.000000
wealthy_percent_got_shoes         0.026000
wealthy_shoes_bought             36.000000
wealthy_shoes_per_person          0.036000
influencer_num_got_shoes          9.000000
influencer_percent_got_shoes      0.018000
influencer_shoes_bought           9.000000
influencer_shoes_per_person       0.018000
reseller_num_got_shoes          326.000000
reseller_pe

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [7]:
env.restock(num_shoes = 1000)
env.run_invitation_with_gaming()
df = env.get_fake_df()
invite_with_gaming = pd.Series(group_fairness(df), name='invite_with_gaming')
invite_with_gaming

Consumer utility                  0.367014
Seller utility                    0.798039
average_num_got_shoes            37.000000
average_percent_got_shoes         0.004111
average_shoes_bought             55.000000
average_shoes_per_person          0.006111
abnormal_num_got_shoes            7.000000
abnormal_percent_got_shoes        0.007000
abnormal_shoes_bought            11.000000
abnormal_shoes_per_person         0.011000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes            14.000000
wealthy_percent_got_shoes         0.014000
wealthy_shoes_bought             20.000000
wealthy_shoes_per_person          0.020000
influencer_num_got_shoes        500.000000
influencer_percent_got_shoes      1.000000
influencer_shoes_bought         500.000000
influencer_shoes_per_person       1.000000
reseller_num_got_shoes          207.000000
reseller_pe

## Change cap to 1
Change the cap to one to prevent resellers get too many shoes and ensure the fairness

In [12]:
env.restock(1000)
env.run_lottery_with_gaming(cap=1)  # run lottery and get dataframe of all results
df = env.get_fake_df()
lottery_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_cap1_with_gaming')

## Authentication

With auth, it's harder for reseller get fake acccounts

In [13]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=2)  # run lottery and get dataframe of all results
df = env.get_fake_df_auth()
lottery_authen_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_with_gaming')

In [14]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=1)  # run lottery and get dataframe of all results
df = env.get_fake_df_auth()
lottery_authen_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_cap1_with_gaming')

## Combine three distribution methods

Since it's easier for resellers to use bot online, shoes will be distributed in a proportion that fewer percentage of them can be bought online to prevent that

In [15]:
env.restock(200)
env.run_lottery_with_gaming_auth()
env.restock_without_reset(500)
env.run_first_come_with_gaming()
env.restock_without_reset(300)
env.run_invitation_with_gaming()
df = env.get_real_fake_df()
combined_distribution_with_gaming = pd.Series(group_fairness(df), name='combined_distribution_with_gaming')

## Get Final Results

In [16]:
final_result = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming,lottery_with_gaming,first_with_gaming,invite_with_gaming,lottery_cap1_with_gaming,lottery_authen_with_gaming,lottery_authen_cap1_with_gaming,combined_distribution_with_gaming])

In [22]:
# uncomment to save results
# final_result.to_csv('project2_results.txt',sep = '\t') 
final_result

,Consumer utility,Seller utility,average_num_got_shoes,average_percent_got_shoes,average_shoes_bought,average_shoes_per_person,abnormal_num_got_shoes,abnormal_percent_got_shoes,abnormal_shoes_bought,abnormal_shoes_per_person,...,wealthy_shoes_bought,wealthy_shoes_per_person,influencer_num_got_shoes,influencer_percent_got_shoes,influencer_shoes_bought,influencer_shoes_per_person,reseller_num_got_shoes,reseller_percent_got_shoes,reseller_shoes_bought,reseller_shoes_per_person
lottery_no_gaming,0.387709,0.166144,383.0,0.042556,568.0,0.063111,55.0,0.055,75.0,0.075,...,56.0,0.056,23.0,0.046,23.0,0.046,128.0,0.0512,256.0,0.1024
first_no_gaming,0.900863,0.083208,516.0,0.057333,787.0,0.087444,52.0,0.052,68.0,0.068,...,76.0,0.076,31.0,0.062,31.0,0.062,0.0,0.0000,0.0,0.0000
invite_no_gaming,0.448519,0.692164,168.0,0.018667,255.0,0.028333,17.0,0.017,26.0,0.026,...,43.0,0.043,500.0,1.000,500.0,1.000,82.0,0.0328,164.0,0.0656
lottery_with_gaming,0.090209,0.428298,63.0,0.007000,98.0,0.010889,13.0,0.013,20.0,0.020,...,9.0,0.009,4.0,0.008,4.0,0.008,432.0,0.1728,863.0,0.3452
first_with_gaming,0.439315,0.312611,200.0,0.022222,313.0,0.034778,19.0,0.019,21.0,0.021,...,22.0,0.022,14.0,0.028,14.0,0.028,309.0,0.1236,618.0,0.2472
invite_with_gaming,0.390938,0.762613,85.0,0.009444,128.0,0.014222,6.0,0.006,8.0,0.008,...,12.0,0.012,500.0,1.000,504.0,1.008,171.0,0.0684,342.0,0.1368
lottery_cap1_with_gaming,0.084370,0.429648,133.0,0.014778,133.0,0.014778,12.0,0.012,12.0,0.012,...,17.0,0.017,5.0,0.010,5.0,0.010,828.0,0.3312,828.0,0.3312
lottery_authen_with_gaming,0.186077,0.342105,156.0,0.017333,236.0,0.026222,17.0,0.017,24.0,0.024,...,30.0,0.030,7.0,0.014,7.0,0.014,344.0,0.1376,687.0,0.2748
lottery_authen_cap1_with_gaming,0.180251,0.344097,286.0,0.031778,286.0,0.031778,35.0,0.035,35.0,0.035,...,30.0,0.030,12.0,0.024,12.0,0.024,628.0,0.2512,628.0,0.2512
combined_distribution_with_gaming,0.422213,0.619708,124.0,0.013778,187.0,0.020778,10.0,0.010,13.0,0.013,...,13.0,0.013,304.0,0.608,311.0,0.622,232.0,0.0928,464.0,0.1856
